<a href="https://colab.research.google.com/github/lukehartfield/Recommender-System/blob/main/Recommender%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analytics for Unstructured Data - Assignment 2
##Group Members -
1. Mihir Jeshurun Gandham
2. Abhilash Bagde
3. Arturo Juarez Jr
4. Vedant Mehrotra
5. Luke Hartfield
6. Bakr Katkhuda



In [ ]:
# @title
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

  Using cached selenium-4.35.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached selenium-4.35.0-py3-none-any.whl (9.6 MB)
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached typing_extensions-4.14.1-py3-none-any.whl (43 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://cli.github.com/packages stable InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,690 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,577 kB]
Get:14 http:

In [ ]:
!pip install gensim

# Task A

Approach for scraping:

 We had done some initial analysis on the Top 250 beers in https://www.beeradvocate.com/beer/top-rated/ but noticed that some of the beers had less than 5 reviews. We would have had to scrape extra reviews from some of the other more reviewed beers which would have skewed the representation of beers. Instead, we explored the following link: https://www.beeradvocate.com/beer/popular/ to scrape 250 beers. Since these were the most popular beers in terms of number of reviews and ratings, we were able to get 35 reviews per beer. This allowed us to get a more accurate representation of the word associations and sentiments for each beer.

In [ ]:
# @title
import os
import re
import csv
import time
import math
import random
from typing import List, Dict
from urllib.parse import urljoin, urlparse, parse_qs, urlencode, urlunparse

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    StaleElementReferenceException,
    ElementClickInterceptedException,
)

# -----------------------------
# Config
# -----------------------------
POPULAR_URL = "https://www.beeradvocate.com/beer/popular/"
LOGIN_URL = "https://www.beeradvocate.com/community/login/"
OUT_CSV = "beer_reviews.csv"
TARGET_BEERS = 250
REVIEWS_PER_BEER = 35

# Flip to False if you want to watch the browser during debug
HEADLESS = True

# ---- HARD-CODED LOGIN (per your request) ----
USER_NAME = "vedant97"
PASSWORD = "unstructured2025!"

# polite delays
def human_delay(a=0.8, b=1.8):
    time.sleep(random.uniform(a, b))

def longer_delay(a=2.0, b=3.5):
    time.sleep(random.uniform(a, b))

# -----------------------------
# Selenium setup
# -----------------------------
def make_driver(headless: bool = HEADLESS) -> webdriver.Chrome:
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--window-size=1400,1000")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # Realistic UA reduces headless/CAPTCHA friction
    options.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
    )
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(60)
    return driver

# -----------------------------
# Small helpers for page readiness / consent
# -----------------------------
def _wait_dom_ready(driver, timeout=30):
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )

def _maybe_close_consent(driver):
    candidates = [
        "//button[contains(., 'Accept')]",
        "//button[contains(., 'I agree')]",
        "//a[contains(., 'Accept')]",
        "//button[contains(@class,'accept')]",
    ]
    for xp in candidates:
        try:
            btn = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.XPATH, xp))
            )
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.5)
            break
        except Exception:
            continue

# -----------------------------
# Auth (BeerAdvocate-specific; hardened)
# -----------------------------
def login(driver, user, pwd):
    wait_long  = WebDriverWait(driver, 45)
    wait_short = WebDriverWait(driver, 8)

    def go(url):
        driver.get(url)
        try:
            wait_short.until(lambda d: d.execute_script("return document.readyState") == "complete")
        except TimeoutException:
            pass
        time.sleep(1.0)

    def first_visible(candidates):
        last_err = None
        for by, sel in candidates:
            try:
                els = driver.find_elements(by, sel)
                for el in els:
                    if el.is_displayed():
                        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                        return el
            except Exception as e:
                last_err = e
        raise TimeoutException(f"No visible match for {candidates}. Last error: {last_err}")

    def js_click(el):
        try:
            el.click()
        except Exception:
            driver.execute_script("arguments[0].click();", el)

    for url in [
        "https://www.beeradvocate.com/community/login/login",
        "https://www.beeradvocate.com/community/login/",
    ]:
        go(url)

        try:
            radio = first_visible([
                (By.CSS_SELECTOR, "#pageLogin input[name='register'][value='0']"),
                (By.ID, "ctrl_pageLogin_registered"),
                (By.ID, "ctrl_registered"),
                (By.XPATH, "//*[@name='register' and (@id='ctrl_registered' or @id='ctrl_pageLogin_registered' or @value='0')]"),
            ])
            if hasattr(radio, "is_selected") and not radio.is_selected():
                js_click(radio)
        except Exception:
            pass

        try:
            user_el = first_visible([
                (By.ID, "LoginControl"),
                (By.ID, "ctrl_pageLogin_login"),
                (By.CSS_SELECTOR, "#pageLogin input[name='login']"),
                (By.NAME, "login"),
            ])
            pass_el = first_visible([
                (By.ID, "ctrl_pageLogin_password"),
                (By.ID, "ctrl_password"),
                (By.CSS_SELECTOR, "#pageLogin input[name='password']"),
                (By.NAME, "password"),
            ])
            user_el.clear(); user_el.send_keys(user)
            pass_el.clear(); pass_el.send_keys(pwd)

            submit = first_visible([
                (By.CSS_SELECTOR, "#pageLogin input[type='submit']"),
                (By.XPATH, "//*[@id='pageLogin']//input[@type='submit' or @value='Log in' or @value='Login']"),
                (By.XPATH, "//button[@type='submit' or contains(., 'Log in') or contains(., 'Login')]"),
            ])
            js_click(submit)

            try:
                wait_long.until(EC.url_changes(url))
            except TimeoutException:
                wait_long.until_not(EC.presence_of_element_located((By.ID, "pageLogin")))
            time.sleep(0.6)
            return
        except Exception:
            continue

    raise TimeoutException("Could not complete login (no visible inputs found).")

# -----------------------------
# Collect beer links (from /beer/popular/)
# Only real beer detail pages: /beer/profile/<brewery_id>/<beer_id>/
# -----------------------------
def collect_beer_links(driver: webdriver.Chrome, limit: int = TARGET_BEERS) -> List[str]:
    driver.get(POPULAR_URL)
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    human_delay()

    anchors = driver.find_elements(By.CSS_SELECTOR, "a[href^='/beer/profile/']")
    links, seen = [], set()

    for a in anchors:
        href = a.get_attribute("href") or ""
        if href.startswith("/"):
            href = urljoin("https://www.beeradvocate.com", href)

        if not re.search(r"/beer/profile/(\d+)/(\d+)(/|$)", href):
            continue

        if href not in seen:
            seen.add(href)
            links.append(href)
            if len(links) >= limit:
                break

    print(f"Collected {len(links)} beer links.")
    return links

# -----------------------------
# Utilities
# -----------------------------
def extract_text_safe(el) -> str:
    try:
        txt = el.text or ""
        return re.sub(r"\s+", " ", txt).strip()
    except Exception:
        return ""

def parse_rating_from_text(txt: str) -> float:
    m = re.search(r"(\d+(?:\.\d+)?)", txt)
    if not m:
        return float("nan")
    try:
        return float(m.group(1))
    except Exception:
        return float("nan")

def get_product_name(driver: webdriver.Chrome) -> str:
    try:
        h1 = driver.find_element(By.XPATH, "//h1")
        name = extract_text_safe(h1)
        if name:
            return name
    except NoSuchElementException:
        pass
    title = driver.title or ""
    title = re.sub(r"\s*\|\s*BeerAdvocate.*$", "", title).strip()
    return title or "Unknown"

def _with_view_ratings(u: str) -> str:
    pu = urlparse(u)
    qs = parse_qs(pu.query, keep_blank_values=True)
    qs["view"] = ["ratings"]
    newq = urlencode(qs, doseq=True)
    return urlunparse((pu.scheme, pu.netloc, pu.path, pu.params, newq, pu.fragment))

def _non_empty_review(txt: str, min_len: int = 10) -> bool:
    if not txt:
        return False
    t = re.sub(r"\s+", " ", txt).strip()
    if not t:
        return False
    return len(t) >= min_len

# -----------------------------
# Scrape reviews (forces ratings view; pairs content/body; skips empty)
# -----------------------------
def scrape_reviews_for_beer(driver: webdriver.Chrome, beer_url: str, max_reviews: int = REVIEWS_PER_BEER) -> List[Dict[str, str]]:
    """
    Collect up to `max_reviews` text reviews from a beer page by paging through
    the "Recent ratings and reviews" list. A valid text review must have BOTH:
      - span.BAscore_norm (the BA normalized score), and
      - an inline-styled <div> body under #rating_fullview_content_2
    Ratings-only rows are skipped.
    """
    out: List[Dict[str, str]] = []
    seen_texts = set()

    def clean(s: str) -> str:
        return re.sub(r"\s+", " ", (s or "").replace("\u00A0", " ")).strip()

    def is_non_trivial(txt: str, min_chars: int = 20, min_words: int = 4) -> bool:
        txt = clean(txt)
        if not txt:
            return False
        if txt.lower().startswith("rated:"):
            return False
        return len(txt) >= min_chars and len(txt.split()) >= min_words

    # Force canonical without query/fragment; BA defaults to "Recent" list
    pu = urlparse(beer_url)
    current_url = urlunparse((pu.scheme, pu.netloc, pu.path, pu.params, "", ""))

    product_name = ""
    fetched_pages = 0

    while len(out) < max_reviews:
        driver.get(current_url)
        # Soft wait on DOM ready
        end = time.time() + 10
        while time.time() < end:
            try:
                if driver.execute_script("return document.readyState") == "complete":
                    break
            except Exception:
                pass
            time.sleep(0.1)
        human_delay(0.6, 1.2)

        if not product_name:
            product_name = get_product_name(driver) or "Unknown"

        # Each review block is a sibling .user-comment with ba-user attr
        blocks = driver.find_elements(By.CSS_SELECTOR, "div.user-comment[ba-user]")
        if not blocks:
            # No blocks at all? Bail.
            break

        for block in blocks:
            # Scope to the content column for this block
            try:
                content = block.find_element(By.XPATH, ".//div[@id='rating_fullview_content_2']")
            except NoSuchElementException:
                continue

            # Require BA normalized score (this excludes "Rated:" rows)
            try:
                score_el = content.find_element(By.CSS_SELECTOR, "span.BAscore_norm")
                ba_norm_score = clean(score_el.get_attribute("textContent") or score_el.text)
            except NoSuchElementException:
                continue

            # Grab the styled body div directly under content_2
            text_div = None
            for xp in [
                ".//div[contains(@style,'font-size') and contains(@style,'line-height')]",
                ".//div[contains(@style,'margin') and contains(@style,'font-size')]",
            ]:
                try:
                    cand = content.find_element(By.XPATH, xp)
                    if cand:
                        text_div = cand
                        break
                except NoSuchElementException:
                    continue
            if not text_div:
                continue

            review_text = (text_div.get_attribute("textContent") or text_div.text or "").strip()
            if not is_non_trivial(review_text):
                continue

            # De-dup by normalized body text
            key = clean(review_text).lower()
            if key in seen_texts:
                continue
            seen_texts.add(key)

            out.append({
                "product_name": product_name,
                "product_review": review_text.strip(),
                "user_rating": ba_norm_score,  # keep as string (e.g., "4.25")
            })

            if len(out) >= max_reviews:
                break

        if len(out) >= max_reviews:
            break

        # Find & follow the "next" page (uses ?start=20 increments)
        next_url = _find_next_page_url(driver)
        if not next_url:
            break

        current_url = next_url
        fetched_pages += 1
        # Soft politeness
        human_delay(0.8, 1.6)

    return out[:max_reviews]

def _find_next_page_url(driver) -> str:
    """
    Return absolute URL for the 'next' page in the Recent ratings/reviews pager,
    or '' if there isn't one.
    """
    try:
        # Prefer a visible 'next' link that advances ?start=
        candidates = driver.find_elements(
            By.XPATH,
            "//a[contains(@href,'start=') and (contains(., 'next') or contains(., 'Next'))]"
        )
        for a in candidates:
            if a.is_displayed():
                href = a.get_attribute("href") or ""
                if href:
                    return urljoin(driver.current_url, href)
    except Exception:
        pass
    return ""


# -----------------------------
# Main Orchestration
# -----------------------------
def main():
    if not USER_NAME or not PASSWORD:
        raise RuntimeError("Hard-coded credentials missing.")

    driver = make_driver(headless=HEADLESS)

    try:
        login(driver, USER_NAME, PASSWORD)
        print("Logged in.")

        beer_links = collect_beer_links(driver, limit=TARGET_BEERS)
        if len(beer_links) < TARGET_BEERS:
            print(f"Warning: only found {len(beer_links)} beer links on the page.")

        if not os.path.exists(OUT_CSV):
            with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
                writer = csv.DictWriter(f, fieldnames=["product_name", "product_review", "user_rating"])
                writer.writeheader()

        for idx, link in enumerate(beer_links, start=1):
            print(f"[{idx}/{len(beer_links)}] {link}")
            try:
                reviews = scrape_reviews_for_beer(driver, link, max_reviews=REVIEWS_PER_BEER)
                print(f"  -> harvested {len(reviews)} reviews")
                if reviews:
                    with open(OUT_CSV, "a", newline="", encoding="utf-8") as f:
                        writer = csv.DictWriter(f, fieldnames=["product_name", "product_review", "user_rating"])
                        for row in reviews:
                            writer.writerow(row)
                longer_delay(2.5, 4.5)
            except TimeoutException:
                print("  !! timeout on this beer, skipping")
                continue
            except Exception as e:
                print(f"  !! error: {e}")
                continue

        print(f"Done. Wrote to {OUT_CSV}")

    finally:
        driver.quit()

if __name__ == "__main__":
    main()


#Task B

In [ ]:
import re
import numpy as np
import pandas as pd
from collections import Counter

# Text / NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Vectorization & similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sentiment
from nltk.sentiment import SentimentIntensityAnalyzer

# Plotting (optional; for inspecting lift as a heatmap)
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
beer = pd.read_csv('beer_reviews (1).csv')

# Download resources
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# --- Clean review text ---
def clean_text(text):
    if pd.isna(text):
        return ""
    return " ".join(re.findall(r"[A-Za-z]+", text)).lower()

beer["review_words"] = beer["product_review"].apply(clean_text)

# --- Tokenize + clean ---
def tokenize(text):
    if pd.isna(text):
        return []
    tokens = re.findall(r"[a-zA-Z]+", text.lower())
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]  # remove stopwords + short words
    tokens = [lemmatizer.lemmatize(t) for t in tokens]  # normalize
    return tokens

beer["tokens"] = beer["review_words"].apply(tokenize)

# Flatten for frequency counts
all_tokens = [token for tokens in beer["tokens"] for token in tokens]

# Top 50 words (more than 25 to spot useful attributes)
freq = Counter(all_tokens)
freq_df = pd.DataFrame(freq.most_common(50), columns=["word", "count"])
print("\n=== Frequency of ALL the words in the reviews (minus stopwords)===")
print(freq_df.head(20))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



=== Frequency of ALL the words in the reviews (minus stopwords)===
           word  count
0          beer   7354
1         taste   5325
2          head   5306
3          malt   4157
4           hop   3636
5        flavor   3378
6          dark   3314
7         aroma   3222
8          like   3108
9          good   3059
10        light   3057
11        sweet   2908
12          one   2794
13         nice   2685
14       finish   2678
15       medium   2652
16         note   2565
17  carbonation   2382
18        smell   2322
19    chocolate   2299


In [ ]:
import pandas as pd
import re
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer

# Make sure resources are downloaded
nltk.download("wordnet")
nltk.download("omw-1.4")

# -----------------------------
# Word Frequency list
# -----------------------------
freq_df = pd.DataFrame(freq.most_common(50), columns=["word", "count"])

# -----------------------------
# Define synonym dictionary for beer attributes
# -----------------------------
ATTRIBUTE_SYNONYMS = {
    # Flavour-related
    "citrusy": "citrus",
    "citrus": "citrus",
    "flavourful": "flavor",
    "flavorful": "flavor",
    "flavors": "flavor",
    "flavour": "flavor",
    "flavor": "flavor",

    # Sweetness
    "sweetness": "sweet",
    "sweet": "sweet",

    # Hop / Malt
    "hoppiness": "hoppy",
    "hoppy": "hoppy",
    "maltyness": "malty",
    "malty": "malty",
    "bready": "bread",
    "bread": "bread",

    # Mouthfeel
    "smoothness": "smooth",
    "smooth": "smooth",
    "bitterness": "bitter",
    "bitter": "bitter",
    "dryness": "dry",
    "dry": "dry",
}

# -----------------------------
# Normalization function
# -----------------------------
lemmatizer = WordNetLemmatizer()

def normalize_attribute(word):
    word = word.lower()
    # First lemmatize (general cleanup)
    lemma = lemmatizer.lemmatize(word)
    # Then apply custom beer-specific mapping
    return ATTRIBUTE_SYNONYMS.get(lemma, lemma)

# -----------------------------
# Apply normalization to tokenized reviews
# -----------------------------
beer["normalized_tokens"] = beer["tokens"].apply(lambda toks: [normalize_attribute(w) for w in toks])

# Flatten again
all_normalized_tokens = [w for toks in beer["normalized_tokens"] for w in toks]

# -----------------------------
# Recompute frequency counts
# -----------------------------
norm_freq = Counter(all_normalized_tokens)
norm_freq_df = pd.DataFrame(norm_freq.most_common(50), columns=["word", "count"])
print("\n=== Normalized Attribute Frequency Table ===")
print(norm_freq_df.head(20))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



=== Normalized Attribute Frequency Table ===
           word  count
0          beer   7354
1         taste   5325
2          head   5306
3         sweet   4195
4          malt   4157
5        bitter   4050
6        flavor   3775
7           hop   3636
8          dark   3314
9         aroma   3222
10         like   3108
11         good   3059
12        light   3057
13          one   2794
14         nice   2685
15       finish   2678
16       medium   2652
17         note   2565
18  carbonation   2382
19        smell   2322


In [ ]:
import nltk
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger_eng")

from nltk import pos_tag, word_tokenize

# -----------------------------
# Identify adjectives
# -----------------------------
def keep_adjectives(words):
    tagged = pos_tag(words)
    return [word for word, pos in tagged if pos.startswith("JJ")]  # JJ = adjective tags

# Apply on normalized tokens per review
beer["adj_tokens"] = beer["normalized_tokens"].apply(keep_adjectives)

# Flatten adjective tokens across all reviews
all_adj_tokens = [w for toks in beer["adj_tokens"] for w in toks]

# -----------------------------
# Frequency of adjectives only
# -----------------------------
adj_freq = Counter(all_adj_tokens)
adj_freq_df = pd.DataFrame(adj_freq.most_common(150), columns=["attribute", "count"])

print("\n=== Top Descriptive Attributes (Adjectives Only) ===")
print(adj_freq_df.head(20))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!



=== Top Descriptive Attributes (Adjectives Only) ===
   attribute  count
0      sweet   3191
1       good   3006
2       nice   2545
3     bitter   2444
4      white   2117
5      light   1992
6     finish   1941
7    overall   1885
8       dark   1618
9      great   1594
10    little   1486
11     black   1295
12       dry   1278
13    strong   1178
14      nose   1111
15     clear   1094
16     brown   1086
17      full   1048
18      much    885
19    smooth    868


In [ ]:
BEER_ATTRIBUTE_LEXICON = {
    "sweet", "light", "dark", "bitter", "strong", "nose", "hoppy", "malt",
    "floral", "creamy", "smooth", "golden", "moderate", "earthy", "thick",
    "aroma", "complex", "sour", "delicious", "mild", "tropical",
    "herbal", "citrusy", "roasty", "aggressive", "crisp", "bread", "caramel", "old", "dry"
}

# -----------------------------
# Filter adjectives to keep only beer-relevant ones
# -----------------------------
filtered_attr_df = adj_freq_df[adj_freq_df["attribute"].isin(BEER_ATTRIBUTE_LEXICON)]

print("\n=== Top Beer-Relevant Attributes ===")
print(filtered_attr_df.sort_values("count", ascending=False).head(15))


=== Top Beer-Relevant Attributes ===
   attribute  count
0      sweet   3191
3     bitter   2444
5      light   1992
8       dark   1618
12       dry   1278
13    strong   1178
14      nose   1111
19    smooth    868
20     hoppy    856
22    golden    801
24  moderate    789
26      malt    732
28    floral    715
29    creamy    669
33    earthy    603


In [ ]:
# -----------------------------
# Compute lift ONCE over all candidate attributes
# -----------------------------
# Candidate attribute universe (from your filtered_attr_df)
candidate_attrs = filtered_attr_df["attribute"].dropna().astype(str).unique().tolist()

# Presence per review (binary)
attr_presence_all = pd.DataFrame(0, index=beer.index, columns=candidate_attrs)
for a in candidate_attrs:
    attr_presence_all[a] = beer["normalized_tokens"].apply(lambda toks: int(a in toks))

def compute_lift_matrix(presence_df: pd.DataFrame) -> pd.DataFrame:
    N = len(presence_df)
    pA = presence_df.sum(axis=0) / N
    attrs = list(presence_df.columns)
    lift = pd.DataFrame(index=attrs, columns=attrs, dtype=float)
    for a in attrs:
        A = presence_df[a].astype(bool)
        pa = pA[a]
        for b in attrs:
            if a == b:
                lift.loc[a, b] = np.nan  # hide self-lift
                continue
            B = presence_df[b].astype(bool)
            pb = pA[b]
            if pa == 0 or pb == 0:
                lift.loc[a, b] = np.nan
            else:
                both = (A & B).sum() / N
                lift.loc[a, b] = both / (pa * pb) if (pa * pb) > 0 else np.nan
    return lift

lift_matrix = compute_lift_matrix(attr_presence_all)

In [ ]:
## -----------------------------
# Expand attributes using co-mentions (Approach 2)
# -----------------------------
SEED_COUNT = 6        # how many high-frequency seeds
PARTNERS_PER_SEED = 3 # how many partners per seed
MIN_LIFT = 1.60       # optional: require minimum lift

# 1) Pick seeds by frequency from filtered_attr_df
core_attrs = (
    filtered_attr_df.sort_values("count", ascending=False)
    .head(SEED_COUNT)["attribute"]
    .tolist()
)

print("=== Core Seed Attributes ===")
print(core_attrs)

# 2) For each seed, add high-lift partners
expanded_attrs = set(core_attrs)
lift_records = []

for seed in core_attrs:
    if seed not in lift_matrix.index:
        continue
    partners = (lift_matrix.loc[seed]
                .dropna()
                .sort_values(ascending=False))
    if MIN_LIFT is not None:
        partners = partners[partners >= MIN_LIFT]
    partners = partners.head(PARTNERS_PER_SEED)
    for partner, val in partners.items():
        if partner != seed:
            expanded_attrs.add(partner)
        lift_records.append((seed, partner, round(float(val), 3)))

expanded_attrs = sorted(list(expanded_attrs))  # IMPORTANT: list for vectorizer

print("\n=== Expanded Attribute Lexicon (Seeds + Partners) ===")
print(expanded_attrs)

lift_expanded_df = pd.DataFrame(
    lift_records, columns=["seed_attr", "partner_attr", "lift"]
).sort_values(["seed_attr", "lift"], ascending=[True, False])

print("\n=== Top Co-Mention Lifts for Seed Attributes ===")
print(lift_expanded_df)

=== Core Seed Attributes ===
['sweet', 'bitter', 'light', 'dark', 'dry', 'strong']

=== Expanded Attribute Lexicon (Seeds + Partners) ===
['aggressive', 'bitter', 'bread', 'caramel', 'complex', 'creamy', 'dark', 'dry', 'herbal', 'light', 'moderate', 'roasty', 'strong', 'sweet', 'thick', 'tropical']

=== Top Co-Mention Lifts for Seed Attributes ===
   seed_attr partner_attr   lift
3     bitter   aggressive  2.111
4     bitter     tropical  1.879
5     bitter     moderate  1.804
9       dark       roasty  3.058
10      dark       creamy  1.982
11      dark      caramel  1.819
12       dry       herbal  2.298
13       dry     moderate  2.269
14       dry   aggressive  2.099
6      light   aggressive  2.021
7      light     moderate  1.874
8      light        bread  1.847
15    strong   aggressive  1.784
16    strong        thick  1.751
17    strong      complex  1.687
0      sweet   aggressive  1.766
1      sweet      caramel  1.703
2      sweet     moderate  1.669


In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

# Ensure VADER is available
nltk.download("vader_lexicon")
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# -----------------------------
# Build beer-level docs restricted to expanded attributes
# -----------------------------
top_attrs = expanded_attrs  # now a sorted list

def filter_to_top_attrs(tokens):
    return " ".join([w for w in tokens if w in top_attrs])

beer["attr_only"] = beer["normalized_tokens"].apply(filter_to_top_attrs)

# Aggregate per product (unchanged)
beer_docs = beer.groupby("product_name")["attr_only"].apply(lambda x: " ".join(x)).reset_index()
beer_ratings = beer.groupby("product_name")["user_rating"].mean().reset_index(name="avg_rating")

# Sentiment per review → per product
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()
beer["sentiment"] = beer["product_review"].astype(str).apply(lambda t: sia.polarity_scores(t)["compound"])
sent_mean = beer.groupby("product_name")["sentiment"].mean().reset_index(name="avg_sentiment")

# Merge
beer_data = beer_docs.merge(beer_ratings, on="product_name", how="left") \
                     .merge(sent_mean,   on="product_name", how="left")

# Drop empty docs
beer_data["attr_only"] = beer_data["attr_only"].fillna("").astype(str)
beer_data = beer_data[beer_data["attr_only"].str.strip().str.len() > 0].reset_index(drop=True)

# Vectorize with fixed vocabulary = expanded attributes
vectorizer = CountVectorizer(vocabulary=top_attrs)
X = vectorizer.fit_transform(beer_data["attr_only"])
feature_names = list(vectorizer.get_feature_names_out())

# Helper: min-max normalize safe
def minmax(s: pd.Series):
    s = s.fillna(s.median())
    if s.max() == s.min():
        return pd.Series(np.ones(len(s)), index=s.index)  # neutral if constant
    return (s - s.min()) / (s.max() - s.min())

# -----------------------------
# Recommender + summary
# -----------------------------
def recommend_beers_with_summary(user_attrs, top_n=3, contenders=20, w_rating=0.5, w_sent=0.5):
    """
    user_attrs: list of 3 strings chosen from the Expanded Attribute Set
    final_score = cosine * (1 + w_rating*norm_rating + w_sent*norm_sentiment)
    Returns: (top3_summary_df, top20_df, ranked_df)
    """
    # Keep only attrs that exist in the restricted vocab
    user_attrs_in = [a for a in user_attrs if a in feature_names]
    if len(user_attrs_in) == 0:
        raise ValueError(f"None of the provided attributes are in top-attrs: {user_attrs}")

    # Build user query vector
    user_query = " ".join(user_attrs_in)
    q_vec = vectorizer.transform([user_query])                 # 1 x V
    cosine = cosine_similarity(q_vec, X).flatten()             # N_beers

    # Normalize rating & sentiment
    r_norm = minmax(beer_data["avg_rating"])
    s_norm = minmax(beer_data["avg_sentiment"])

    # Final score
    score = cosine * (1 + w_rating*r_norm + w_sent*s_norm)

    # Pack into results
    res = beer_data.copy()
    res["cosine"] = cosine
    res["rating_norm"] = r_norm
    res["sent_norm"]   = s_norm
    res["score"]       = score

    # Attribute counts per beer (from X)
    counts_df = pd.DataFrame(X.toarray(), columns=feature_names, index=res.index)

    # For interpretability: add per-user-attribute counts
    for a in user_attrs_in:
        res[f"count_{a}"] = counts_df[a]

    # Also add coverage (how many of the user’s attributes the beer actually hits)
    coverage_cols = [f"count_{a}" for a in user_attrs_in]
    res["matched_attrs"] = (res[coverage_cols] > 0).sum(axis=1)
    res["total_hits"]    = res[coverage_cols].sum(axis=1)  # total mentions across chosen attrs

    # Rank
    ranked = res.sort_values(["score","matched_attrs","total_hits"], ascending=[False, False, False]).reset_index(drop=True)

    # Top-3 summary: include interpretability columns
    show_cols = (["product_name","avg_rating","avg_sentiment","cosine","score","matched_attrs","total_hits"]
                 + coverage_cols)
    top3_summary = ranked.loc[:top_n-1, show_cols]

    # Top-20 contenders (core columns)
    top20 = ranked.loc[:contenders-1, ["product_name","avg_rating","avg_sentiment","cosine","score"]]

    return top3_summary, top20, ranked

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

attr_options = sorted(top_attrs)

dropdown1 = widgets.Dropdown(options=attr_options, description='Attribute 1:')
dropdown2 = widgets.Dropdown(options=attr_options, description='Attribute 2:')
dropdown3 = widgets.Dropdown(options=attr_options, description='Attribute 3:')

button = widgets.Button(description="Get Recommendations", button_style='success')
msg = widgets.HTML("")  # for friendly validation messages
output = widgets.Output()

def on_button_click(b):
    global user_attributes   # <-- declare global so it persists outside
    with output:
        clear_output()
        # Collect user inputs
        user_attributes = [dropdown1.value, dropdown2.value, dropdown3.value]

        print(f"Selected attributes: {user_attributes}")

        # Run recommender (BoW)
        top3_summary, top20_table, all_ranked = recommend_beers_with_summary(
            user_attributes, top_n=3, contenders=20
        )

        print("\n=== Top 3 Recommendations (BoW) ===")
        display(top3_summary)

        print("\n=== Top 20 Contenders (BoW) ===")
        display(top20_table)

button.on_click(on_button_click)

display(dropdown1, dropdown2, dropdown3, button, msg, output)

Dropdown(description='Attribute 1:', options=('aggressive', 'bitter', 'bread', 'caramel', 'complex', 'creamy',…

Dropdown(description='Attribute 2:', options=('aggressive', 'bitter', 'bread', 'caramel', 'complex', 'creamy',…

Dropdown(description='Attribute 3:', options=('aggressive', 'bitter', 'bread', 'caramel', 'complex', 'creamy',…

Button(button_style='success', description='Get Recommendations', style=ButtonStyle())

HTML(value='')

Output()

## Task B – Method Explanation  

The goal of Task B is to build the basic foundation of a **crowdsourced beer recommender system** that takes user-specified attributes and suggests beers accordingly.  
Here is the step-by-step methodology we implemented:  

### 1. Attribute Discovery from Reviews  
- We first **tokenized and cleaned** the text of all reviews.  
- Using **part-of-speech tagging**, we extracted adjectives and descriptors (since these most often describe beer properties).  
- From these, we filtered a domain-specific lexicon of beer-relevant attributes (e.g., *sweet, strong, light, malt, hoppy etc*).  
- Instead of relying purely on frequency, we selected **5–6 high-frequency seed attributes** (such as *sweet, strong, light etc*). These served as anchor descriptors.
- From the wide list of adjectives which we collected out of the reviews. As opposed to general-purpose descriptions, such as best, great, and nice (which do not give meaningful clues about beer preferences), we hand-selected a collection of adjectives that describe actual beer-specific qualities. Such as: sweet, bitter, strong, hoppy, malty.

### 2. Attribute Co-Occurrence (Lift Expansion)  
- We computed **lift ratios** to measure co-occurrence between attributes:  
- For each seed attribute, we identified other attributes with **high lift values** (i.e., terms co-mentioned more than expected by chance).  
- This expanded our attribute set beyond frequency, ensuring we captured realistic pairings such as *strong → aggressive*, *sweet → caramel*, and *bitter → roasty*.  
- The final lexicon was therefore **frequency-grounded and association-aware**, reflecting how drinkers naturally describe beers.  

### 3. Beer-Level Aggregation  
- Reviews were aggregated at the **product (beer) level**.  
- For each beer, we created a text “document” containing only mentions of the expanded attributes.  
- We also calculated:  
  - **Average user rating** (numeric score).  
  - **Average sentiment score** using VADER sentiment analysis.  

### 4. Bag-of-Words + Cosine Similarity  
- We used a **CountVectorizer** restricted to the expanded attributes to create a beer × attribute matrix.  
- A **user query** (3 chosen attributes) was represented in the same vector space.  
- **Cosine similarity** was applied to measure closeness between each beer’s profile and the user’s preferences.  

### 5. Final Scoring with Rating & Sentiment  
- To ensure high-quality recommendations, we combined:  
  - **Cosine similarity** (relevance to user attributes).  
  - **Normalized rating** (average user scores).  
  - **Normalized sentiment** (positivity of review text).  
- The final score was defined as:  
  \[
  \text{Final Score} = \text{Cosine Similarity} \times (1 + w_r \times \text{Rating} + w_s \times \text{Sentiment})
  \]  
  where \( w_r \) and \( w_s \) are weights that balance rating and sentiment.  

### 6. Recommendations via Interactive UI  
- Beers were ranked by final score, and we also tracked:  
  - How many of the user’s attributes matched.  
  - How frequently these attributes were mentioned.  
- We built a **dropdown-based interactive interface** (using `ipywidgets`), where users select 3 attributes from the expanded lexicon.  
- The recommender then outputs:  
  - **Top 3 Recommendations** with interpretability columns.  
  - **Top 20 Contenders** for transparency.  


#Task C

In [ ]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 55.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# -----------------------------
# Task C: Word Vectors (spaCy) based recommender
# -----------------------------
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load spaCy medium English model with word vectors
nlp = spacy.load("en_core_web_md")

def doc_embedding(text):
    doc = nlp(text)
    if len(doc) == 0:
        return np.zeros(nlp.vocab.vectors_length)
    return doc.vector

# Build beer-level embeddings (restricted to attr_only from expanded set)
beer_data["embedding"] = beer_data["attr_only"].apply(doc_embedding)
beer_embeddings = np.vstack(beer_data["embedding"].values)

def recommend_beers_with_embeddings(user_attrs, top_n=3, contenders=20, w_rating=0.5, w_sent=0.5):
    # Make sure query is restricted to expanded attributes
    user_attrs_in = [a for a in user_attrs if a in expanded_attrs]
    if len(user_attrs_in) == 0:
        raise ValueError(f"None of the provided attributes are in expanded set: {user_attrs}")

    # Build user query embedding
    user_query = " ".join(user_attrs_in)
    q_vec = doc_embedding(user_query).reshape(1, -1)

    # Cosine similarity
    sims = cosine_similarity(q_vec, beer_embeddings).flatten()

    # Normalize rating & sentiment (same as Task B)
    r_norm = minmax(beer_data["avg_rating"])
    s_norm = minmax(beer_data["avg_sentiment"])
    score = sims * (1 + w_rating*r_norm + w_sent*s_norm)

    res = beer_data.copy()
    res["cosine_emb"] = sims
    res["score_emb"] = score

    ranked = res.sort_values("score_emb", ascending=False).reset_index(drop=True)

    top3 = ranked.loc[:top_n-1, ["product_name","avg_rating","avg_sentiment","cosine_emb","score_emb"]]
    top20 = ranked.loc[:contenders-1, ["product_name","avg_rating","avg_sentiment","cosine_emb","score_emb"]]

    return top3, top20, ranked

In [ ]:
# -----------------------------
# Example Run (same attributes as before)
# -----------------------------

top3_emb, top20_emb, all_ranked_emb = recommend_beers_with_embeddings(user_attributes)

print("\n=== Top 3 with Word Vectors ===")
print(top3_emb)

print("\n=== Top 20 with Word Vectors ===")
print(top20_emb)


=== Top 3 with Word Vectors ===
                                       product_name  avg_rating  \
0  Wisconsin Belgian Red New Glarus Brewing Company    4.449714   
1  Bourbon County Brand Stout Goose Island Beer Co.    4.548286   
2         Raspberry Tart New Glarus Brewing Company    4.448000   

   avg_sentiment  cosine_emb  score_emb  
0       0.876574    0.943537   1.840192  
1       0.824140    0.918881   1.767865  
2       0.777757    0.945012   1.762155  

=== Top 20 with Word Vectors ===
                                         product_name  avg_rating  \
0    Wisconsin Belgian Red New Glarus Brewing Company    4.449714   
1    Bourbon County Brand Stout Goose Island Beer Co.    4.548286   
2           Raspberry Tart New Glarus Brewing Company    4.448000   
3                  Troegenator Tröegs Brewing Company    4.078571   
4             Burton Baton Dogfish Head Craft Brewery    4.322286   
5   CBS (Canadian Breakfast Stout) Founders Brewin...    4.586571   
6          Gu

## Task C – Bag-of-Words vs. Word Vectors

We re-ran the recommendation pipeline using **word embeddings (spaCy medium vectors)** instead of the bag-of-words attribute matrix.

### Bag-of-Words Approach
- Represents each beer as a **sparse vector** of attribute counts (restricted to the attributes previously selected in Part B).
- User query is matched to beers using **cosine similarity** on these vectors.
- Strengths: precise, interpretable (we know which attributes matched).
- Limitation: does not capture synonyms or semantic similarity. For example, *citrusy* and *lemon* are treated as unrelated.

### Word Vector Approach
- Each beer document and user query is represented as the **average of spaCy word embeddings**.
- Cosine similarity compares semantic closeness in vector space.
- Advantage: captures **synonyms and related words** (e.g., *citrusy* ≈ *lemon*, *smooth* ≈ *creamy*).
- Limitation: loses interpretability and may “smooth over” subtle distinctions between attributes.

### Which Works Better?
- In our dataset, most user attributes are **domain-specific adjectives** (hoppy, malty, crisp, smooth).  
- These terms are used **literally and consistently** in reviews, so the **bag-of-words model is more precise** and easier to interpret.  
- Word vectors may help in edge cases (e.g., *citrusy* vs. *lemon*), but overall they introduce noise since many words in beer reviews (e.g., *nice*, *great*, *good*) are semantically close but not discriminative.  

### Conclusion
- For this dataset, **bag-of-words provides better recommendations** because attributes are domain-specific and exact-match is more valuable than semantic fuzziness.  
- Word vectors could improve results in broader domains where synonyms and varied phrasing is common but here precision and interpretability are essential.


In [ ]:
# -----------------------------
# Side-by-side Comparison (Task C)
# -----------------------------

# Run Bag-of-Words recommender
top3_bow, top20_bow, _ = recommend_beers_with_summary(user_attributes, top_n=3, contenders=20)

# Run Embedding recommender
top3_emb, top20_emb, _ = recommend_beers_with_embeddings(user_attributes, top_n=3, contenders=20)

# Format side-by-side table
comparison = pd.DataFrame({
    "Rank": [1, 2, 3],
    "BOW_Top3": top3_bow["product_name"].tolist(),
    "Emb_Top3": top3_emb["product_name"].tolist()
})

print("\n=== Side-by-Side Comparison (Top 3) ===")
print(comparison)

# If you also want to see top-20 overlap:
bow_set = set(top20_bow["product_name"])
emb_set = set(top20_emb["product_name"])
overlap = bow_set & emb_set

print("\n=== Overlap in Top-20 between BOW and Embeddings ===")
print(f"Overlap count: {len(overlap)}")
print(sorted(list(overlap))[:10])  # show up to 10 names


=== Side-by-Side Comparison (Top 3) ===
   Rank                                 BOW_Top3  \
0     1  Burton Baton Dogfish Head Craft Brewery   
1     2    Hop Rod Rye Bear Republic Brewing Co.   
2     3       Troegenator Tröegs Brewing Company   

                                           Emb_Top3  
0  Wisconsin Belgian Red New Glarus Brewing Company  
1  Bourbon County Brand Stout Goose Island Beer Co.  
2         Raspberry Tart New Glarus Brewing Company  

=== Overlap in Top-20 between BOW and Embeddings ===
Overlap count: 11
['Backwoods Bastard Founders Brewing Company', "Brown Shugga' Lagunitas Brewing Company", 'Burton Baton Dogfish Head Craft Brewery', 'Gulden Draak Brouwerij Van Steenberge N.V.', 'Hop Rod Rye Bear Republic Brewing Co.', "Hopslam Ale Bell's Brewery - Eccentric Café & General Store", 'Hoptimum Sierra Nevada Brewing Co.', 'Maharaja Avery Brewing Company', 'Raspberry Tart New Glarus Brewing Company', 'Troegenator Tröegs Brewing Company']


## Task C – Why Do the Two Lists Differ?

After running both methods, we compared the **Top-3 and Top-20 recommendations** from the Bag-of-Words (BOW) model and the Word Vector (Embedding) model.  
The rankings differ because of how each method represents and interprets the review text:

### 1. Bag-of-Words (BOW)
- Each beer is represented by **explicit counts** of the 18 attributes (sweet, aggressive, bitter, etc.).  
- Cosine similarity matches only on **exact word overlap**.  
- This makes BOW **precise and interpretable**: if a beer appears in the Top-3, it is because reviewers explicitly used those same words.  
- Limitation: cannot connect synonyms or related words (e.g., *citrusy* vs. *lemon*).

### 2. Word Vectors (Embeddings)
- Each beer document and user query are converted into **dense semantic vectors** (via spaCy embeddings).  
- Cosine similarity measures **semantic closeness**, so beers can rank highly even if they don’t use the exact user words.  
  - Example: A beer described as *citrusy* may be retrieved for a query containing *lemon* or *fruity*.  
- Advantage: captures broader meaning and synonyms.  
- Limitation: may recommend beers that match **general sentiment words** (e.g., *good*, *nice*, *delicious*) that don’t reflect true attribute alignment.

### 3. Why Results Differ
- **BOW tends to reward beers** that reviewers consistently describe with the **exact same terms** as the user query.  
- **Embeddings may pull in beers** that use **related or similar words**, broadening the set but sometimes introducing noise.  
- As a result, overlap in the Top-20 lists is partial: some beers are consistently strong candidates, while others differ because embeddings consider indirect associations.

### 4. Which Is Better in This Context?
- In beer reviews, attributes like *sweet, moderate, aggresive, light* are **domain-specific and consistently used**.  
- Thus, **BOW usually gives more precise and reliable recommendations**.  
- Word vectors could be more useful if reviewers used **more varied language** or lots of synonyms to describe beers.  


**Conclusion:**  
The two lists differ because **BOW prioritizes literal matches**, while **embeddings prioritize semantic similarity**.  
For this dataset, BOW is more effective, but embeddings highlight possible **“hidden” matches** that could be interesting for broader discovery.


# Task D

In [ ]:
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

def recommend_beers_with_embeddings_tfidf(user_attributes, beer_df,
                                          token_col='normalized_tokens',
                                          attr_col='attr_only',
                                          rating_col='user_rating',
                                          sentiment_col='sentiment',
                                          vector_size=100,
                                          window=5,
                                          min_count=5):
    """
    Recommend beers using custom Word2Vec embeddings weighted by TF-IDF.

    Parameters
    ----------
    user_attributes : list of str
        Attributes chosen by the user.
    beer_df : pd.DataFrame
        DataFrame containing tokenized reviews, attribute-only text, ratings, and sentiment.
        Expected to have columns like 'product_name', 'normalized_tokens', 'attr_only',
        'user_rating', 'sentiment'.
    token_col : str
        Column with list-of-tokens per review (for training Word2Vec).
    attr_col : str
        Column with attribute-only text per review.
    rating_col, sentiment_col : str
        Columns with user ratings and sentiment per review.
    vector_size, window, min_count : int
        Word2Vec hyperparameters.

    Returns
    -------
    top3_emb : pd.DataFrame
        Top 3 beer recommendations with aggregated info.
    top20_emb : pd.DataFrame
        Top 20 beer recommendations with aggregated info.
    all_ranked_emb : pd.DataFrame
        Full DataFrame with scores and aggregated info.
    """
    # --- 1. Aggregate review data by product name ---
    # Aggregate tokens for Word2Vec training and attr_only for TF-IDF
    # Also calculate average rating and sentiment per product
    beer_grouped = beer_df.groupby('product_name').agg(
        tokens=(token_col, lambda x: [token for sublist in x if isinstance(sublist, list) for token in sublist]),
        attr_only=(attr_col, lambda x: ' '.join(x.dropna().astype(str))),
        avg_rating=(rating_col, 'mean'),
        avg_sentiment=(sentiment_col, 'mean')
    ).reset_index()

    # Filter out beers with no attribute text after aggregation
    beer_grouped = beer_grouped[beer_grouped['attr_only'].str.strip().str.len() > 0].reset_index(drop=True)


    # --- 2. Train Word2Vec on aggregated tokens ---
    sentences = beer_grouped['tokens'].tolist()
    w2v_model = Word2Vec(
        sentences,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        workers=4,
        sg=1
    )

    # --- 3. Fit TF-IDF on aggregated attribute text ---
    corpus = beer_grouped['attr_only'].tolist()
    tfidf = TfidfVectorizer(tokenizer=str.split)
    tfidf.fit(corpus)
    idf_scores = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

    # --- 4. Helper: weighted document vector using TF-IDF ---
    def doc_vector_tfidf(tokens, model, idf_scores):
        vecs = []
        weights = []
        # Ensure tokens is iterable (list or string) and not empty/NaN
        if not isinstance(tokens, (list, str)) or (isinstance(tokens, str) and not tokens.strip()):
             return np.zeros(model.vector_size)

        # If tokens is a string, split it
        if isinstance(tokens, str):
            tokens = tokens.split()

        for w in tokens:
            if w in model.wv and w in idf_scores:
                vecs.append(model.wv[w] * idf_scores[w])
                weights.append(idf_scores[w])
        if not vecs:
            return np.zeros(model.vector_size)
        return np.sum(vecs, axis=0) / (np.sum(weights) + 1e-9) # Add small epsilon to avoid division by zero

    # --- 5. Build per-beer vectors from aggregated attr_only text ---
    beer_vectors = []
    for txt in beer_grouped['attr_only'].tolist():
         toks = txt.split()
         beer_vectors.append(doc_vector_tfidf(toks, w2v_model, idf_scores))
    beer_vectors = np.vstack(beer_vectors)


    # --- 6. Build user query vector ---
    user_toks = [t for t in user_attributes]
    q_vec = doc_vector_tfidf(user_toks, w2v_model, idf_scores)

    # --- 7. Cosine similarity ---
    if np.all(q_vec == 0): # Handle case where no user attributes are in the vocabulary
        cosine = np.zeros(len(beer_vectors))
    else:
        q_vec = q_vec.reshape(1, -1)
        cosine = cosine_similarity(q_vec, beer_vectors).flatten()


    # --- 8. Normalize rating and sentiment ---
    def minmax(x):
        x = np.array(x)
        if np.all(np.isnan(x)) or (np.nanmax(x) == np.nanmin(x)):
             return np.zeros_like(x, dtype=float)
        if np.any(np.isnan(x)):
             return (x - np.nanmin(x)) / (np.nanmax(x) - np.nanmin(x))
        else:
            return (x - x.min()) / (x.max() - x.min())

    r_norm = minmax(beer_grouped['avg_rating'])
    s_norm = minmax(beer_grouped['avg_sentiment'])


    # --- 9. Final score ---
    score = cosine * (1 + 0.5*r_norm + 0.5*s_norm)

    # --- 10. Attach to DataFrame ---
    ranked = beer_grouped.copy()
    ranked['cosine_emb_tfidf'] = cosine
    ranked['score_emb_tfidf']  = score
    ranked = ranked.sort_values('score_emb_tfidf', ascending=False).reset_index(drop=True)

    # --- 11. Select and return top results ---
    show_cols = ["product_name", "avg_rating", "avg_sentiment", "cosine_emb_tfidf", "score_emb_tfidf"]
    top3_emb = ranked[show_cols].head(3)

    return top3_emb, ranked

In [ ]:
# --- Example run ---

top3, all_ranked = recommend_beers_with_embeddings_tfidf(
    user_attributes,
    beer,  # your original raw review DataFrame
    token_col='normalized_tokens',
    attr_col='attr_only',
    rating_col='user_rating',
    sentiment_col='sentiment'
)

print("=== Top 3 Beer Recommendations ===")
print(top3)


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


=== Top 3 Beer Recommendations ===
                                        product_name  avg_rating  \
0  CBS (Canadian Breakfast Stout) Founders Brewin...    4.586571   
1         Gulden Draak Brouwerij Van Steenberge N.V.    4.304000   
2            Burton Baton Dogfish Head Craft Brewery    4.322286   

   avg_sentiment  cosine_emb_tfidf  score_emb_tfidf  
0       0.856529          0.918406         1.799361  
1       0.838471          0.939712         1.775601  
2       0.785026          0.961398         1.775594  


### Do Our results change when using custom word embeddings?

Considering the top 3 reccomendations extracted from the custom word embedding model above, we see some new beers that have not previously been among the top 3 reccomendations enter into the mix. However, when looking at the top 20 reccomendations from both part B and part C, some of these new reccomendations were included in the top 20 in both previous parts. So, although they werent among the strongest recommendations in the previous parts, they have consistently been ranked high enough, and it seems our custom Word2Vec embeddings are highlighting semantic relationships in the reviews that werent fully captured by the default SpaCy embeddings or simpler models. This shows that while the overall set of “highly recommended” beers remains largely consistent, the custom embeddings allow for finer differentiation among beers, especially when customer preferences are expressed as specific attributes.

# Task E

In [ ]:
top_rated_beers = beer.groupby('product_name')['user_rating'].mean().sort_values(ascending=False).head(3)
print(top_rated_beers)


product_name
Pliny The Younger Russian River Brewing Company                                              4.709143
Pliny The Elder Russian River Brewing Company                                                4.675714
Trappist Westvleteren 12 (XII) Brouwerij Westvleteren (Sint-Sixtusabdij van Westvleteren)    4.633429
Name: user_rating, dtype: float64


In [ ]:
from collections import Counter
from itertools import chain

# --- 1. Get top 3 rated beers ---
top3_rated = beer.groupby('product_name')['user_rating'].mean().sort_values(ascending=False).head(3)
top3_rated_names = top3_rated.index.tolist()

# --- 2. Calculate lift for each beer individually ---
lift_results = {}

# Compute overall frequency of user attributes in the full dataset
all_adj_tokens = list(chain.from_iterable(beer['adj_tokens']))
overall_counts = Counter([w for w in all_adj_tokens if w in user_attributes])
overall_total = len(all_adj_tokens)
overall_freq = {attr: overall_counts.get(attr, 0)/overall_total for attr in user_attributes}

for beer_name in top3_rated_names:
    # Extract adjectives for this beer
    beer_adj_tokens = list(chain.from_iterable(beer[beer['product_name'] == beer_name]['adj_tokens']))
    beer_total = len(beer_adj_tokens)
    beer_counts = Counter([w for w in beer_adj_tokens if w in user_attributes])

    # Compute lift for each attribute
    lift = {}
    for attr in user_attributes:
        beer_freq = beer_counts.get(attr, 0) / beer_total if beer_total > 0 else 0
        overall_freq_attr = overall_freq.get(attr, 1e-9)  # avoid divide by zero
        lift[attr] = beer_freq / overall_freq_attr
    lift_results[beer_name] = lift

# --- 3. Display results ---
import pandas as pd
lift_df = pd.DataFrame(lift_results).T
print("Lift scores for top 3 rated beers (user attributes):")
print(lift_df)


Lift scores for top 3 rated beers (user attributes):
                                                       sweet   caramel  \
Pliny The Younger Russian River Brewing Company     0.684454  0.000000   
Pliny The Elder Russian River Brewing Company       0.354902  1.093139   
Trappist Westvleteren 12 (XII) Brouwerij Westvl...  0.715923  1.470084   

                                                      creamy  
Pliny The Younger Russian River Brewing Company     0.725492  
Pliny The Elder Russian River Brewing Company       0.000000  
Trappist Westvleteren 12 (XII) Brouwerij Westvl...  1.422839  


To assess whether the top 3 highest-rated beers would meet the users preferences, we first selected the beers with the highest average ratings across the dataset. For each beer, we extracted the adjectives from its reviews and calculated lift scores for the previously selected attributes, "sweet", "creamy", and "caramel". Lift measures how much more (or less) frequently an attribute appears in the beers reviews compared to the dataset overall. A lift greater than 1 indicates that the attribute is strongly associated with the beer, while a lift less than 1 suggests the attribute is underrepresented and not characteristic of that beer.

Examining the lift scores we obtained on the three highest rated beers, we see that most attributes have values below 1 for these top-rated beers, indicating that these beers are not strongly associated with certain user-preferred characteristics. For instance, a beer may be highly rated overall but not particularly "sweet" or "creamy". This shows that simply choosing the highest-rated products does not necessarily satisfy the users specific flavor preferences, as ratings reflect overall popularity rather than alignment with the desired attributes. Therefore, while these beers might be enjoyable in general, they would not reliably meet the requirements of a user seeking beers with the particular combination of "sweet", "creamy", and "caramel" flavors.

# Task F

In [ ]:
# ---------- parameters ----------
SAMPLE_N = 10
RANDOM_STATE = 40
TARGET_BEER = None   # set to a product_name if you want to force the target

# ---------- helpers ----------
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def relfreq_vec(tokens, vocab_list):
    c = Counter(tokens)
    total = sum(c.values()) or 1
    return np.array([c[w] / total for w in vocab_list], dtype=float)

# ---------- aggregate per beer ----------
# attr_only is a space-separated string of adjectives, so split them into tokens
beer["__attr_tokens"] = beer["attr_only"].apply(lambda x: x.split() if isinstance(x, str) else [])

tokens_by_beer = (
    beer.groupby("product_name")["__attr_tokens"]
    .apply(lambda rows: [w for toks in rows for w in toks])
)

# sample 10 beers (ensure target included if specified)
if TARGET_BEER and TARGET_BEER in tokens_by_beer.index:
    others = tokens_by_beer.drop(index=TARGET_BEER).sample(SAMPLE_N - 1, random_state=RANDOM_STATE)
    sample_tokens = pd.concat([tokens_by_beer.loc[[TARGET_BEER]], others])
else:
    sample_tokens = tokens_by_beer.sample(SAMPLE_N, random_state=RANDOM_STATE)

sample_names = sample_tokens.index.tolist()
target_beer = TARGET_BEER if (TARGET_BEER in sample_names) else sample_names[0]

# vocab just from these 10
vocab = sorted(set(w for toks in sample_tokens for w in toks))

# build beer × vocab matrix
X = np.vstack([relfreq_vec(sample_tokens.loc[name], vocab) for name in sample_names])
beer_mat = pd.DataFrame(X, index=sample_names, columns=vocab)

# cosine similarity
S = cosine_similarity(beer_mat.values)
S_df = pd.DataFrame(S, index=sample_names, columns=sample_names)

# nearest neighbor for the target
s_target = S_df.loc[target_beer].drop(target_beer)
most_similar = s_target.idxmax()
best_score = s_target.max()

print("=== Task F (attr_only) Result ===")
print(f"Target beer:           {target_beer}")
print(f"Most similar (of 9):   {most_similar}")
print(f"Cosine similarity:     {best_score:.3f}")

# rationale: top shared adjectives
v1 = beer_mat.loc[target_beer]
v2 = beer_mat.loc[most_similar]
top_shared = ((v1 + v2) / 2).sort_values(ascending=False).head(10)

print("\nTop shared adjectives (by avg relative frequency):")
print(top_shared.to_frame("avg_rel_freq").round(4))

print("\nSimilarity to the other 9 beers:")
print(s_target.sort_values(ascending=False).round(3).to_frame("cosine"))


=== Task F (attr_only) Result ===
Target beer:           Dale's Pale Ale Oskar Blues Grill & Brew
Most similar (of 9):   Arctic Panzer Wolf 3 Floyds Brewing Co.
Cosine similarity:     0.926

Top shared adjectives (by avg relative frequency):
          avg_rel_freq
bitter          0.2487
sweet           0.1839
dry             0.0778
caramel         0.0751
bread           0.0673
light           0.0564
dark            0.0541
thick           0.0422
moderate        0.0387
creamy          0.0377

Similarity to the other 9 beers:
                                                    cosine
Arctic Panzer Wolf 3 Floyds Brewing Co.              0.926
Fresh Squeezed IPA Deschutes Brewery                 0.919
Weihenstephaner Vitus Bayerische Staatsbrauerei...   0.869
Abbey Ale Brewery Ommegang                           0.807
Pumking Southern Tier Brewing Company                0.797
Shakespeare Oatmeal Stout Rogue Ales                 0.789
2XIPA Southern Tier Brewing Company                  0.775

For Task F, We used the attr_only adjectives from reviews we had already created (e.g., bitter, sweet, dry) to represent each beer as a vector of relative attribute frequencies, so beers with longer reviews wouldn’t dominate. I then compared these vectors using cosine similarity, which measures how closely two descriptive profiles align regardless of scale. Among ten sampled beers, Dale’s Pale Ale was most similar to Arctic Panzer Wolf (cosine = 0.926), with both frequently described as bitter and sweet, plus secondary notes like dry and caramel. Changing the random seed yields different sets of beers, but the closest pairs almost always belong to the same style, showing the method captures meaningful style-based similarities.